# Stochastic variational inference

Assume the target variable is normally distributed around some mean function of the input, given by a neural network, with some variance.

\begin{equation*}
p(y | x, \theta) := \mathcal{N}(\mu = NN(x, \theta), K(y, y') = 1)
\end{equation*}

Put a prior on the parameters of the network

\begin{equation*}
p(\theta) := \mathcal{N}(\mu = 0, \sigma^2 = 100 \mathcal{I})
\end{equation*}

and try to infer their posterior.

As variational distribution, use an independent (mean-field VI) multivariate Gaussian

\begin{equation*}
q_\phi(\theta) := \mathcal{N}(\mu_q, \sigma_q^2 \mathcal{I})
\end{equation*}

where $\phi := (\mu_q,\sigma_q)$ are the variational parameters, which are randomly initialised.

We then optimise a sample-based approximation of the evidence lower bound (the KL divergence between the variational distribution and the posterior, with the evidence $p(y)$ dropped as it doesn' depend on $\theta$.

\begin{equation*}
\mathbb{E}_{\theta \sim q_\phi}\left[ \log\frac{q_\phi(\theta)}{p(y | x, \theta)p(\theta)} \right]
\approx \frac{1}{S}\sum_{i=1}^S \log\frac{q_\phi(\theta^{(i)})}{p(y | x, \theta^{(i)})p(\theta^{(i)})}\,,
\end{equation*}

with respect to the variational parameters, where $\{\theta^{(i)}\}_{i=1}^S\overset{i.i.d.}{\sim} q_\phi$.

That was surprisingly easy.  Proper initialisation still helps a lot I think.  But yeah, actually seems kind of easier than gradient descent..?  

Seems to fit better with a prior.  Similar to regularisation right.

Seems like the output distribution is just a sum of normals so I can probably find it analytically instead of sampling.  Would be cool to check how the variance develops moving away from the data too.  And explore how the prior affects the fit.

I should try to show how using the full posterior as opposed to just point estimates avoids overfitting.

In [ ]:
# Import packages
import numpy as np
import torch
import plotly.graph_objects as go
from Code.vi import *
from Code.tools import *

In [ ]:
# Make data
x = torch.linspace(-1, 1, 101).reshape(101, 1)

# Target functions, from easy to hard
# y = torch.maximum(2 * x, torch.zeros_like(x))
# y = x**3
# y = np.exp(x)
y = np.sin(np.pi * x)

# Set number of neurons in hidden layer
n_hidden=20

# Initialise variational parameters
q_mean = init_par(n_hidden, x, y)
plot_neurons(q_mean, x, y, n_hidden, title="Initial neuron outputs")
log_q_sd = torch.log(torch.rand([3 * n_hidden + 1]))
log_q_sd.requires_grad = True

# Run variational inference
var_inf(
    q_mean, log_q_sd, x, y, n_hidden, n_iterations = int(2e3), plot_every = 100, 
    n_samples = 20, lr = 1e-2
)

In [ ]:
# Run variational inference
var_inf(
    q_mean, log_q_sd, x, y, n_hidden, n_iterations = int(2e3), plot_every = 100, 
    n_samples = 20, lr = 1e-2
)